In [3]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import creep_event_picker as cep
import obspy
from tqdm.notebook import tqdm
import h5py
import re
from scipy import stats

How many events are at each creepmeter?

In [4]:
Creepmeter_dataframe = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes_updated.csv',index_col=0)
Creepmeter_dataframe.drop_duplicates('Creepmeter_abbrv',inplace=True)
Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe['Creepmeter_abbrv']=='XMBC'].index,inplace=True)
Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe['Creepmeter_abbrv']=='TABC'].index,inplace=True)
Creepmeter_dataframe.reset_index(inplace=True,drop=True)
Creepmeter_dataframe

,Network,Creepmeter_abbrv,File_code,Start Time,End Time,"Sampling rate, Hz","Sampling rate, mins",Latitude,Longitude,Length,Depth,Obliquity,Creepmeter_full_name,Fault
0,EAF,BAL1,BAL1_1T,2023-03-12T10:47:55.000000Z,2023-09-05T13:14:55.000000Z,0.016667,1T,37.9903,38.1990,5 m,NaN,30.0,Balikburnu,East Anatolian Fault
1,PARK,C461,C461_10T,2004-09-29T02:20:00.000000Z,2022-06-20T12:40:00.000000Z,0.001667,10T,35.7240,-120.2820,NaN,NaN,30.0,Highway 46,San Andraes Fault - Parkfield
2,PARK,C462,C462_1T,2021-08-19T19:10:11.000000Z,2024-03-22T01:05:11.000000Z,0.016667,1T,35.7240,-120.2820,NaN,NaN,30.0,Highway 46,San Andraes Fault - Parkfield
3,UTA,CAN1,CAN1_30T,2012-06-12T15:33:00.000000Z,2017-06-15T08:33:00.000000Z,0.000556,30T,38.0198,-110.0374,7.5 m,NaN,45.0,Canyonlands,Southern Imperial Valley Fault - Utah
4,NAF,CER1,CER1_30S,2019-10-09T06:46:00.000000Z,2024-05-23T10:25:30.000000Z,0.033333,30S,40.8947,32.7773,20 m,NaN,29.0,Cerkes,North Anatolian Fault
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,EAF,YZW1,YZW1_1T,2023-03-22T10:59:37.000000Z,2024-05-26T14:10:37.000000Z,0.016667,1T,38.1086,38.7361,5 m,NaN,35.0,Yazica West,East Anatolian Fault
74,SOCAL,COML,COML_30T,2010-08-15T00:30:00,2011-02-03T16:55:05,0.000556,30T,33.8965,-117.0866,NaN,NaN,30.0,Mystic Lake,San Jacinto Fault
75,SOCAL,COLE,COLE_30T,2010-08-10T07:01:56,2011-02-20T20:31:56,0.000556,30T,32.6643,-115.8790,NaN,NaN,30.0,Yuha East,Laguna Salada Fault
76,SOCAL,COLW,COLW_5T,2010-04-10T00:10:00,2010-04-16T17:40:00,0.003333,5T,32.6483,-115.8480,NaN,NaN,30.0,Yuha West,Laguna Salada Fault


In [5]:
frequency_dict = {
        1/60: '1S',    # 1 second
        1/6: '10S',  # 10 seconds
        1/2: '30S',  # 30 seconds
        1: '1T',  # 1 minute
        2: '2T',  # 2 minutes
        5: '5T',  # 5 minutes
        10: '10T',# 10 minutes
        15: '15T', # 15 minutes
        30: '30T',# 30 minutes
        60: '60T'    # 60 minutes
    }

In [6]:
#Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe['Creepmeter_abbrv']!='SC30'].index,inplace=True)

In [23]:
Columns = ['Network','Creepmeter full name','Creepmeter abbreviation','Latitude','Longitude',\
           'Sampling rate, s','Start Time','End Time','Displacement, mm','Duration, hrs','Maximum velocity, m/s','Time to next event, days']
df_all = pd.DataFrame(columns=Columns)
e_id = 0
left_to_qc = 0
for i in tqdm(range(len(Creepmeter_dataframe))):
    creep_ortho = []
    tm_ortho = []
    if Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i] in ['FCR1','GOK1','KAR1','DS30','SC30']:
        with h5py.File('../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i]), 'r') as f:
            for key in f.keys():
                if key in ['Orthogonal']:
                    data = f[key]  # Access the dataset
                    data_keys = list(data.keys())
                    #print(data_keys)
                    slip_ortho = data[data_keys[0]][:]
                    time_ortho =  data[data_keys[1]][:]
                    decoded_time = [byte_str.decode('utf-8') for byte_str in time_ortho]
                    decoded_time = pd.to_datetime(decoded_time)
                    sample_rate_ortho = (decoded_time[1]-decoded_time[0])/dt.timedelta(minutes=1)
                    smpl_rate = frequency_dict[sample_rate_ortho]


        tm_ortho, creep_ortho, upsampled = cep.interpolate_linear(decoded_time,slip_ortho,smpl_rate)  

   
    print(Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i])
    
    with h5py.File('../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i]), 'r') as f:
        # Loop through each key in the file
        keys = list(f.keys())
        f.attrs
        for key in f.keys():
            starts = []
            ends = []
            event_slip = []  
            event_duration = []
            event_sample_rate = []
            event_velocity = []
            days_list = []
            if key not in ['Temperature','Temperature_1T','Temperature_5T','Daily_measurements','Manual_measurements','Orthogonal']:
                #print("Key:", key)
                #print('extracting data')
                data = f[key]  # Access the dataset
                data_keys = list(data.keys())
                #print(data_keys)
                slip = data[data_keys[0]][:]
                time =  data[data_keys[1]][:]
                decoded_time = [byte_str.decode('utf-8') for byte_str in time]
                decoded_time = pd.to_datetime(decoded_time)

                match = re.search(r'_(\d*\.?\d+)mins$', key)
                if match:
                    freq_value = float(match.group(1))
                    new_suffix = frequency_dict.get(freq_value)
                    if new_suffix:
                        new_key = re.sub(r'_\d*\.?\d+mins$', f'_{new_suffix}', key)
                    else:
                        new_key = key  # or raise an error / warning
                else:
                    new_key = key  # unchanged if pattern not matched

                print(new_key)
                try:
                    df_picks = pd.read_csv("../../Data/DATA_tidied/Picks/Final/{q}_picks_final.csv".format(q =new_key),index_col=0)
                except FileNotFoundError:
                    print('no events')
                    continue

                print("../../Data/DATA_tidied/Picks/Final/{q}_picks_final.csv".format(q =new_key))
                #print(len(df_picks))
                df_picks['Longitude'] = [Creepmeter_dataframe['Longitude'].iloc[i]]*len(df_picks)
                df_picks['Latitude'] = [Creepmeter_dataframe['Latitude'].iloc[i]]*len(df_picks)
                df_picks['Network'] = [Creepmeter_dataframe['Network'].iloc[i]]*len(df_picks)
                df_picks['Creepmeter full name'] = [Creepmeter_dataframe['Creepmeter_full_name'].iloc[i]]*len(df_picks)
                df_picks['Creepmeter abbreviation'] = [Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i]]*len(df_picks)
                #df_picks['Sampling rate, mins'] = [f[key].attrs['sampling_rate']]*len(df_picks)
                obliquity = f.attrs['obliquity']
                
                

                
                
                Data_fr = pd.DataFrame({'Time':decoded_time,'Slip':slip})

                Data_fr.drop_duplicates(subset='Time',inplace=True)
                Data_fr.reset_index(inplace=True,drop=True)


                tm_int = Data_fr['Time']
                creep_int = Data_fr['Slip']

                if Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i] not in ['CFW1','CHP1','COZ1','CPP1','CRR1','CTM1','CWC3','CWN1','SJN1','TABC',
                                                                    'WKR1','X461','XGH1','XHR1','XHR2','XHR3','XHSW','XMD1','XMM1','XMR1',
                                                                    'XPK1','XPK2','XRSW','XSC1','XSH1','XSJ2','XSJ3','XTA1','XVA1']:
                    '''if Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i] == 'SC30':
                        sample_rate = frequency_dict[sample_rate]#Creepmeter_dataframe['Sampling rate, mins'].iloc[i]

            
                        tm_int, creep_int, upsampled = cep.interpolate_linear(tm_dt,creep,sample_rate)
                        tm_ortho, creep_ortho, upsampled = cep.interpolate_linear(decoded_time,slip_ortho,sample_rate=sample_rate)
                        
                        creep_int  = creep_int - creep_int[0]
                        if sample_rate =='1T':
                            record_cut = np.where(pd.to_datetime(tm_ortho)==pd.to_datetime(tm_int[0]))[0][0]
                            creep_ortho = creep_ortho[record_cut:]
                            tm_ortho = tm_ortho[record_cut:]
                        elif sample_rate == '5T':
                            record_cut = np.where(pd.to_datetime(tm_ortho)==pd.to_datetime(tm_int[-1]))[0][0]
                            creep_ortho = creep_ortho[:record_cut+1]
                            tm_ortho = tm_ortho[:record_cut+1]
                        
                            creep_ortho = creep_ortho - creep_ortho[0]
                            
                            creep_int_cor = fault_parallel_from_two_rods_array(creep_int,obliquity,creep_ortho,76)
                        else:
                            creep_int_cor = (creep_int)/np.cos(np.deg2rad(obliquity))
                    
                    
                    
                    
                    else:
                        creep_int = (creep_int)/np.cos(np.deg2rad(obliquity))'''
                    
                    creep_int = (creep_int)/np.cos(np.deg2rad(obliquity))

                
                try:
                    df_picks.drop(df_picks[df_picks['Quality_check_final'] != 'CE'].index,inplace=True)
                    #print('final used')
                except:
                    df_picks.drop(df_picks[df_picks['Quality_check_new'] != 'CE'].index,inplace=True)
                    #print('new used')
                df_picks.reset_index(inplace=True,drop=True)

                try:
                    start_times = pd.to_datetime(df_picks['ST_final'])
                    time_deltas = start_times.diff().iloc[1:]  # Skip the first NaT

                    # Convert timedelta to float days
                    days_as_floats = time_deltas.dt.total_seconds() / (24 * 60 * 60)

                    # Convert to list
                    days_list = days_as_floats.tolist()
                    print(days_list)

                    days_list.insert(0,"First Event")
                except:
                    dummy=10
                
                for j in tqdm(range(len(df_picks))):
                    # Find the index of the closest datetime
                    nearest_s = tm_int[np.argmin(np.abs(np.array(tm_int) - np.datetime64(df_picks.ST_final.iloc[j])))]
                    nearest_e = tm_int[np.argmin(np.abs(np.array(tm_int) - np.datetime64(df_picks.ET_final.iloc[j])))]
                    '''try:
                        nearest_s = tm_int[np.argmin(np.abs(np.array(tm_int) - np.datetime64(df_picks.ST_final.iloc[j])))]
                        nearest_e = tm_int[np.argmin(np.abs(np.array(tm_int) - np.datetime64(df_picks.ET_final.iloc[j])))]
                        
                    except:
                        nearest_s = tm_int[np.argmin(np.abs(np.array(tm_int) - np.datetime64(df_picks.ST_new.iloc[j])))]
                        nearest_e = tm_int[np.argmin(np.abs(np.array(tm_int) - np.datetime64(df_picks.ET_new.iloc[j])))]'''
                        
                    
                    starts.append(nearest_s)
                    ends.append(nearest_e)
                    boolarr = np.logical_and(np.array(tm_int)>=nearest_s,np.array(tm_int)<=nearest_e)
                    event_creep = creep_int[boolarr].to_numpy()
                    event_time = tm_int[boolarr].to_numpy()

                    sample_rate_s = np.median(np.diff(event_time))/np.timedelta64(1,'s')
                    velocity_m_s = (np.max(np.diff(event_creep))/1000)/sample_rate_s
                    
                    event_slip.append(event_creep[-1] - event_creep[0])
                    event_duration.append((pd.to_datetime(event_time[-1]) - pd.to_datetime(event_time[0]))/dt.timedelta(hours=1))
                    event_sample_rate.append(sample_rate_s)
                    event_velocity.append(velocity_m_s)

                df_picks['Displacement, mm'] = event_slip
                df_picks['Duration, hrs'] = event_duration
                df_picks['Start Time'] = starts
                df_picks['End Time'] = ends
                df_picks['Sampling rate, s'] = event_sample_rate
                df_picks['Maximum velocity, m/s'] = event_velocity
                df_picks['Time to next event, days'] = days_list
                
                before = len(df_all)
                df_all = pd.concat([df_all, df_picks[Columns]], ignore_index=True)
                after = len(df_all)
                #print(f"Rows before: {before}, Rows after: {after}, Rows added: {after-before}")
print('Left to QC:',left_to_qc)

  0%|          | 0/78 [00:00<?, ?it/s]

BAL1
BAL1_1T
../../Data/DATA_tidied/Picks/Final/BAL1_1T_picks_final.csv
[6.085416666666666, 155.16482638888888, 0.5215277777777778, 0.15069444444444444, 0.19236111111111112, 0.4013888888888889, 0.0625, 0.2625]


  0%|          | 0/9 [00:00<?, ?it/s]

C461


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_10180/2753248908.py:187: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all, df_picks[Columns]], ignore_index=True)


...

COML
COML_30T
no events
COLE
COLE_30T
no events
COLW
COLW_30T
../../Data/DATA_tidied/Picks/Final/COLW_30T_picks_final.csv
[]


  0%|          | 0/1 [00:00<?, ?it/s]

COLW_5T
no events
EASH
EASH_1T
../../Data/DATA_tidied/Picks/Final/EASH_1T_picks_final.csv
[]


  0%|          | 0/1 [00:00<?, ?it/s]

EASH_10T
no events
EASH_5T
no events
Normal
no events
Left to QC: 0


In [24]:
event_creep

array([-1.06843461, -1.06810228, -1.06776995, ..., -1.03952207,
       -1.03952207, -1.03952207])

In [25]:
df_picks

,ST_new,ET_new,Quality_check_new,ST,ET,Quality_check,ST_final,ET_final,Quality_check_final,Longitude,...,Network,Creepmeter full name,Creepmeter abbreviation,"Displacement, mm","Duration, hrs",Start Time,End Time,"Sampling rate, s","Maximum velocity, m/s","Time to next event, days"
0,2021-07-25 08:17:00,2021-07-27 18:58:00,CE,2021-07-25 08:17:00,2021-07-27 18:58:00,CE,2021-07-25 08:17:15,2021-07-27 18:58:15,CE,-115.7801,...,SOCAL,East Shoreline Normal,EASH,0.028913,58.683333,2021-07-25 08:17:15,2021-07-27 18:58:15,60.0,1.661640e-08,First Event


In [26]:
nearest_s

Timestamp('2021-07-25 08:17:15')

In [27]:
nearest_e

Timestamp('2021-07-27 18:58:15')

In [28]:
#df_all.dropna(inplace=True)
df_all.sort_values(by='Start Time',inplace=True)
df_all.reset_index(inplace=True,drop=True)
df_all

,Network,Creepmeter full name,Creepmeter abbreviation,Latitude,Longitude,"Sampling rate, s",Start Time,End Time,"Displacement, mm","Duration, hrs","Maximum velocity, m/s","Time to next event, days"
0,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,600.0,1980-02-19 14:30:00,1980-02-20 21:29:59,0.190000,30.999722,2.333333e-08,First Event
1,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,600.0,1980-02-21 06:29:59,1980-02-23 08:10:00,0.282000,49.666944,3.500000e-08,1.666655
2,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,600.0,1980-07-22 19:59:59,1980-07-24 19:30:00,0.211000,47.500278,2.500000e-08,152.5625
3,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,600.0,1980-09-10 09:09:59,1980-09-12 03:29:59,1.063000,42.333333,5.166667e-07,49.548611
4,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,600.0,1980-12-13 12:20:00,1980-12-13 20:50:00,0.127000,8.500000,4.833333e-08,94.131956
...,...,...,...,...,...,...,...,...,...,...,...,...
5871,HAY,Fremont Winery,CFW1,37.5320,-121.9520,60.0,2024-02-04 10:03:59,2024-02-05 22:43:59,0.102000,36.666667,3.333333e-08,0.727766
5872,HOL,Fox Creek,FCR1,36.6925,-121.3061,60.0,2024-02-12 15:27:00,2024-02-13 05:30:00,0.027713,14.050000,1.732051e-08,10.840972
5873,HOL,Fox Creek,FCR1,36.6925,-121.3061,60.0,2024-02-16 06:00:00,2024-02-22 11:33:00,4.274586,149.550000,3.271652e-07,3.60625
5874,HOL,Fox Creek,FCR1,36.6925,-121.3061,60.0,2024-02-23 13:55:00,2024-02-28 06:04:00,0.208423,112.150000,2.501851e-08,7.329861


In [29]:
event_id = []
for i in range(len(df_all)):
    event_id.append("{:04d}".format(i) + '_{k}'.format(k=df_all['Creepmeter abbreviation'].iloc[i]))
df_all['Event_id'] = event_id
df_all

,Network,Creepmeter full name,Creepmeter abbreviation,Latitude,Longitude,"Sampling rate, s",Start Time,End Time,"Displacement, mm","Duration, hrs","Maximum velocity, m/s","Time to next event, days",Event_id
0,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,600.0,1980-02-19 14:30:00,1980-02-20 21:29:59,0.190000,30.999722,2.333333e-08,First Event,0000_XMR1
1,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,600.0,1980-02-21 06:29:59,1980-02-23 08:10:00,0.282000,49.666944,3.500000e-08,1.666655,0001_XMR1
2,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,600.0,1980-07-22 19:59:59,1980-07-24 19:30:00,0.211000,47.500278,2.500000e-08,152.5625,0002_XMR1
3,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,600.0,1980-09-10 09:09:59,1980-09-12 03:29:59,1.063000,42.333333,5.166667e-07,49.548611,0003_XMR1
4,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,600.0,1980-12-13 12:20:00,1980-12-13 20:50:00,0.127000,8.500000,4.833333e-08,94.131956,0004_XMR1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5871,HAY,Fremont Winery,CFW1,37.5320,-121.9520,60.0,2024-02-04 10:03:59,2024-02-05 22:43:59,0.102000,36.666667,3.333333e-08,0.727766,5871_CFW1
5872,HOL,Fox Creek,FCR1,36.6925,-121.3061,60.0,2024-02-12 15:27:00,2024-02-13 05:30:00,0.027713,14.050000,1.732051e-08,10.840972,5872_FCR1
5873,HOL,Fox Creek,FCR1,36.6925,-121.3061,60.0,2024-02-16 06:00:00,2024-02-22 11:33:00,4.274586,149.550000,3.271652e-07,3.60625,5873_FCR1
5874,HOL,Fox Creek,FCR1,36.6925,-121.3061,60.0,2024-02-23 13:55:00,2024-02-28 06:04:00,0.208423,112.150000,2.501851e-08,7.329861,5874_FCR1


In [33]:
df_all.dropna(inplace=True)

In [34]:
df_all.to_csv("../../Data/DATA_tidied/Picks/All_picks_12_JUN_2025.csv")

Testing for adding in orthogonal component

In [ ]:
def fault_parallel_from_two_rods_array(d1, theta1_deg, d2, theta2_deg):
    """
    Vectorised version to compute fault-parallel slip from two rods over a time series.

    d1, d2: arrays of displacement time series (same shape)
    theta1_deg, theta2_deg: angles from fault-parallel (scalars, in degrees)
    
    Returns: array of s_parallel (same shape as d1 and d2)
    """
    theta1 = np.radians(theta1_deg)
    theta2 = np.radians(theta2_deg)

    sin1 = np.sin(theta1)
    sin2 = np.sin(theta2)
    cos1 = np.cos(theta1)
    cos2 = np.cos(theta2)

    numerator = d2 * sin1 - d1 * sin2
    denominator = cos2 * sin1 - cos1 * sin2

    if np.isclose(denominator, 0).any():
        raise ValueError("Denominator close to zero — angles may be degenerate or too similar.")

    s_parallel = numerator / denominator
    return s_parallel

In [ ]:
abbrv = 'FCR1'

In [ ]:
if abbrv in ['FCR1','GOK1','KAR1','DS30','SC30']:
    with h5py.File('../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=abbrv), 'r') as f:
        for key in f.keys():
            if key in ['Orthogonal']:
                data = f[key]  # Access the dataset
                data_keys = list(data.keys())
                print(data_keys)
                slip_ortho = data[data_keys[0]][:]
                time_ortho =  data[data_keys[1]][:]
                decoded_time = [byte_str.decode('utf-8') for byte_str in time_ortho]
                decoded_time = pd.to_datetime(decoded_time)


    

    path = "../../Data/DATA_tidied/SAC/{q}.SAC".format(q='FCR1_1T')#Creepmeter_dataframe.File_code.iloc[i])
    st = obspy.read(path)
    tr = st[0]
    tr.interpolate(sampling_rate=tr.stats.sampling_rate)
    tm = tr.times("utcdatetime")
    creep = tr.data
    sample_rate =  round(tr.stats.delta)/60
    tm_dt = []
    for w in range(len(tm)):
        tm_new = tm[w].datetime
        tm_dt.append(tm_new)

    sample_rate = frequency_dict[sample_rate]#Creepmeter_dataframe['Sampling rate, mins'].iloc[i]

    
    tm_int, creep_int, upsampled = cep.interpolate_linear(tm_dt,creep,sample_rate)
    tm_ortho, creep_ortho, upsampled = cep.interpolate_linear(decoded_time,slip_ortho,sample_rate=sample_rate)
    
    creep_int  = creep_int - creep_int[0]
    creep_ortho = creep_ortho[:-1] - creep_ortho[0]
    creep_int_cor = fault_parallel_from_two_rods_array(creep_int,30,creep_ortho,90)

In [ ]:
record_cut

In [ ]:
plt.figure()
plt.plot(tm_ortho, creep_ortho)
plt.plot(tm_int,creep_int)
plt.plot(tm_int,creep_int_cor)
plt.show()

In [ ]:
tm_int[-1]

In [ ]:
tm_ortho[-1]

In [ ]:
elif 'Quality_check' in df_picks.columns:
            left_to_qc +=1
            #print(Creepmeter_dataframe["File_code"].iloc[i],'old QC')
            df_picks.drop(df_picks[df_picks['Quality_check'] != 'CE'].index,inplace=True)
            df_picks.reset_index(inplace=True,drop=True)
            print(len(df_picks))
            for j in range(len(df_picks)):
                #print(j,df_picks.ST_new.iloc[j],df_picks.ET_new.iloc[j])
                #print('old QC')
                boolarr = np.logical_and(np.array(tm_int)>=np.datetime64(df_picks.ST.iloc[j]),\
                                        np.array(tm_int)<=np.datetime64(df_picks.ET.iloc[j]))
                event_creep = creep_int[boolarr]
                event_time = tm_int[boolarr]
                e_id +=1
                event_code.append('{k}'.format(k=Creepmeter_dataframe.Creepmeter_abbrv.iloc[i]) + "{:04d}".format(e_id))
                event_slip.append(event_creep[-1] - event_creep[0])
                event_duration.append((pd.to_datetime(event_time[-1]) - pd.to_datetime(event_time[0]))/dt.timedelta(hours=1))
            df_picks['Displacement, mm'] = event_slip
            df_picks['Duration, hrs'] = event_duration
            df_picks['Event_id'] = event_code
            df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
            df_picks['ET'] = df_picks['ET'].astype(str).str[0:19]
            df_picks['ST'] = pd.Series(pd.to_datetime(df_picks['ST'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])
            df_picks['ET'] = pd.Series(pd.to_datetime(df_picks['ET'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])
        
        else:
            print(len(df_picks))
            print(Creepmeter_dataframe["File_code"].iloc[i],'no QC')
            left_to_qc +=1
            for j in range(len(df_picks)):
                #print('old QC')
                boolarr = np.logical_and(np.array(tm_int)>=np.datetime64(df_picks.ST.iloc[j]),\
                                        np.array(tm_int)<=np.datetime64(df_picks.ET.iloc[j]))
                event_creep = creep_int[boolarr]
                event_time = tm_int[boolarr]
                e_id +=1
                event_code.append('{k}'.format(k=Creepmeter_dataframe.Creepmeter_abbrv.iloc[i]) + "{:04d}".format(e_id))
                event_slip.append(event_creep[-1] - event_creep[0])
                event_duration.append((pd.to_datetime(event_time[-1]) - pd.to_datetime(event_time[0]))/dt.timedelta(hours=1))
            df_picks['Displacement, mm'] = event_slip
            df_picks['Duration, hrs'] = event_duration
            df_picks['Event_id'] = event_code
            df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
            df_picks['ET'] = df_picks['ET'].astype(str).str[0:19]
            df_picks['ST'] = pd.Series(pd.to_datetime(df_picks['ST'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])
            df_picks['ET'] = pd.Series(pd.to_datetime(df_picks['ET'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])

In [ ]:
%matplotlib qt

In [ ]:
plt.close('all')
with h5py.File('../../Data/DATA_tidied/HDF5/DU30.h5', 'r') as f:
        # Loop through each key in the file
        keys = list(f.keys())
        f.attrs
        for key in f.keys():
            starts = []
            ends = []
            event_slip = []  
            event_duration = []
            #event_code = []
            if key not in ['Temperature','Temperature_1T','Temperature_5T','Daily_measurements','Manual_measurements','Orthogonal']:
                print("Key:", key)
                print('extracting data')
                data = f[key]  # Access the dataset
                data_keys = list(data.keys())
                print(data_keys)
                slip = data[data_keys[0]][:]
                time =  data[data_keys[1]][:]
                decoded_time = [byte_str.decode('utf-8') for byte_str in time]
                decoded_time = pd.to_datetime(decoded_time)
        
            plt.figure()
            plt.plot(decoded_time,slip,label=key)
            plt.legend()
            plt.show()